In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [5]:
import sys
if ('..' not in sys.path):
    sys.path.append('..')

print(len(sys.path))

9


In [6]:
import os
import shutil
import subprocess
# import sparsechem as sc
import numpy as np
import string
import glob
import scipy.sparse
import pandas as pd
import argparse
# from urllib.request import urlretrieve
# import torch 
# from torch.utils.data import DataLoader
# from torch.optim.lr_scheduler import MultiStepLR
# from torchinfo import summary
# from torch.utils.tensorboard import SummaryWriter
import pprint
import time
# from sparsechem.utils import training_arguments, load_task_weights
# from GPUtil import showUtilization as gpu_usage
from datetime import datetime
pp = pprint.PrettyPrinter(indent=4)

In [140]:
# cmd = (
#         f" --x           /home/kbardool/WSL-projs/MLDatasets/chembl23_mini/chembl_23mini_x.npy" +
#         f" --y_class     /home/kbardool/WSL-projs/MLDatasets/chembl23_mini/chembl_23mini_y.npy" +
#         f" --folding     /home/kbardool/WSL-projs/MLDatasets/chembl23_mini/chembl_23mini_folds.npy" +
#         f" --fold_inputs 32000" +
#         f" --output_dir  /home/kbardool/WSL-projs/MLDatasets/chembl23_mini_5tasks" +
#         f" --hidden_sizes 1000 1000 1000 5" )

# cmd = (
#         f" --y_class     /home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_all_bin_sparse")
# parser = training_arguments()        
# args = parser.parse_args(cmd.split())

y_class_path = "/home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_all_bin_sparse.npy"
output_path  = "/home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/"

# ecfp     = sc.load_sparse(args.x)
y_class  = load_sparse(y_class_path)
# y_regr   = sc.load_sparse(args.y_regr)
# y_censor = sc.load_sparse(args.y_censor)
print() 
# vprint(f"ecfp shape             : {ecfp.shape}")
print(f"y_class shape          : {y_class.shape}")
print(type(y_class))
# vprint(f"x shape                : {ecfp.shape}")
# vprint(f"y_class shape          : {y_class.shape}")
# vprint(f"y_regr shape           : {y_regr.shape}")

# vprint(f"y_censor shape         : {y_censor.shape}")


y_class shape          : (18388, 15)
<class 'scipy.sparse.csr.csr_matrix'>


In [62]:
import copy
a = copy.copy(y_class)

In [63]:
print(type(a) )

print(a.shape )
print(a.__dict__)
print(a.indices.shape)
print(' indices / cols : ',a.indices[:32])
print(a.indptr.shape)
print(' indptr  : ',a.indptr[:32])
print(a.data.shape)
print('   data  : ',a.data[:32])

<class 'scipy.sparse.csr.csr_matrix'>
(18388, 15)
{'_shape': (18388, 15), 'maxprint': 50, 'data': array([ 1,  1, -1, ...,  1, -1, -1]), 'indices': array([ 0,  1,  2, ..., 12, 13, 14], dtype=int32), 'indptr': array([     0,     15,     30, ..., 275790, 275805, 275820], dtype=int32)}
(275820,)
 indices / cols :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2  3  4  5  6  7  8
  9 10 11 12 13 14  0  1]
(18389,)
 indptr  :  [  0  15  30  45  60  75  90 105 120 135 150 165 180 195 210 225 240 255
 270 285 300 315 330 345 360 375 390 405 420 435 450 465]
(275820,)
   data  :  [ 1  1 -1 -1 -1 -1  1 -1  1  1 -1  1  1 -1  1  1 -1  1 -1  1  1  1 -1 -1
 -1 -1  1 -1 -1 -1 -1  1]


In [41]:
a[2] = [1,2,3,4,5]

/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
Y1_sparse = scipy.sparse.csr_matrix(Y1_labels)
Y2_sparse = scipy.sparse.csr_matrix(Y2_labels)
Y3_sparse = scipy.sparse.csr_matrix(Y3_labels)
print(type(Y1_sparse))
print(type(Y2_sparse))
print(type(Y3_sparse))

np.save('./output/chembl_23mini_adashare_y1.npy', Y1_np)
np.save('./output/chembl_23mini_adashare_y2.npy', Y2_np)
np.save('./output/chembl_23mini_adashare_y3.npy', Y3_np)
np.save('./output/chembl_23mini_adashare_y1_bin.npy', Y1_labels)
np.save('./output/chembl_23mini_adashare_y2_bin.npy', Y2_labels)
np.save('./output/chembl_23mini_adashare_y3_bin.npy', Y3_labels)
np.save('./output/chembl_23mini_adashare_y1_bin_sparse.npy', Y1_sparse)
np.save('./output/chembl_23mini_adashare_y2_bin_sparse.npy', Y2_sparse)
np.save('./output/chembl_23mini_adashare_y3_bin_sparse.npy', Y3_sparse)

tmp = np.load('./output/chembl_23mini_adashare_y1_bin_sparse.npy', allow_pickle=True).item().tocsr()
print(type(tmp))

In [154]:
def split_csr_matrix_by_cols(mat, indices, pfx = ''):
    start_ind = 0
    sum = 0
    for files, i in enumerate(col_split,1):
        sum += i
        if sum >= a.shape[1]:
            break
     
    print(f" column-wise splitting matrix of shape {mat.shape} into {files} files")
    for idx, end_ind in enumerate(indices,1):
        stop_ind = min(start_ind+end_ind, mat.shape[1])
        print(f" place {stop_ind - start_ind} columns [{start_ind} : {stop_ind}] into file {idx}")
        tmp = mat[:, start_ind:stop_ind]
        print(' tmp shape: ', tmp.shape)
        fn = os.path.join(output_path, f"chembl_23mini_adashare_y_[{start_ind}:{stop_ind}].npy")
        np.save(fn, tmp )
        print(' written to : ', fn)
        start_ind = stop_ind
    if start_ind < mat.shape[1]:
        idx+=1
        print(f" place columns [{start_ind} : {mat.shape[1]}] into file {idx}")
#         fn = os.path.join(output_path, f"chembl_23mini_adashare_y{idx}_{stop_ind - start_ind}cols.npy")        
        fn = os.path.join(output_path, f"chembl_23mini_adashare_y_[{start_ind}:{stop_ind}].npy")
        np.save(fn, tmp )                
        print(' written to : ', fn)


In [156]:
col_split =  [2, 2, 2, 2, 2, 2, 2, 1]
# col_split =  [2,  2,11]
# col_split =  [2, 13]
split_csr_matrix_by_cols(a, col_split)

 column-wise splitting matrix of shape (18388, 15) into 8 files
 place 2 columns [0 : 2] into file 1
 tmp shape:  (18388, 2)
 written to :  /home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_[0:2].npy
 place 2 columns [2 : 4] into file 2
 tmp shape:  (18388, 2)
 written to :  /home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_[2:4].npy
 place 2 columns [4 : 6] into file 3
 tmp shape:  (18388, 2)
 written to :  /home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_[4:6].npy
 place 2 columns [6 : 8] into file 4
 tmp shape:  (18388, 2)
 written to :  /home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_[6:8].npy
 place 2 columns [8 : 10] into file 5
 tmp shape:  (18388, 2)
 written to :  /home/kbardool/WSL-projs/MLDatasets/chembl23_synthetic/chembl_23mini_adashare_y_[8:10].npy
 place 2 columns [10 : 12] into file 6
 tmp shape:  (18388, 2)
 written to :  /home/kbardool/WSL-projs/M

In [129]:
from itertools import accumulate

    

3 15


In [64]:
a_array = a.toarray()

a_array.shape

a_array[:10,:]

array([[ 1,  1, -1, -1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1],
       [ 1, -1,  1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1],
       [-1,  1, -1, -1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1],
       [ 1,  1, -1,  1,  1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1],
       [ 1, -1, -1,  1, -1, -1,  1, -1,  1,  1,  1,  1, -1,  1, -1],
       [ 1, -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1, -1, -1],
       [-1, -1, -1,  1,  1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1],
       [ 1,  1, -1,  1,  1,  1, -1,  1,  1,  1, -1, -1, -1,  1, -1],
       [ 1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1,  1, -1],
       [ 1, -1,  1,  1, -1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1]])

In [1]:
def training_arguments():
    parser = argparse.ArgumentParser(description="Training a multi-task model.")
    parser.add_argument("--x"                 , help="Descriptor file (matrix market, .npy or .npz)", type=str, default=None)
    parser.add_argument("--y_class", "--y", "--y_classification" 
                                              , help="Activity file (matrix market, .npy or .npz)", type=str, default=None)
    parser.add_argument("--y_regr" , "--y_regression"
                                              , help="Activity file (matrix market, .npy or .npz)", type=str, default=None)
    parser.add_argument("--y_censor"          , help="Censor mask for regression (matrix market, .npy or .npz)", type=str, default=None)
    parser.add_argument("--batch_ratio"       , help="Batch ratio", type=float, default=0.02)
    parser.add_argument("--censored_loss"     , help="Whether censored loss is used for training (default 1)", type=int, default=1)
    parser.add_argument("--dev"               , help="Device to use", type=str, default="cuda:0")
    parser.add_argument("--epochs"            , help="Number of epochs", type=int, default=20)
    parser.add_argument("--eval_train"        , help="Set this to 1 to calculate AUCs for train data", type=int, default=0)
    parser.add_argument("--eval_frequency"    , help="The gap between AUC eval (in epochs), -1 means to do an eval at the end.", type=int, default=1)
    parser.add_argument("--folding"           , help="Folding file (npy)", type=str, required=True)
    parser.add_argument("--fold_inputs"       , help="Fold input to a fixed set (default no folding)", type=int, default=None)
    parser.add_argument("--fold_va"           , help="Validation fold number", type=int, default=0)
    parser.add_argument("--fold_te"           , help="Test fold number (removed from dataset)", type=int, default=None)
    parser.add_argument("--hidden_sizes"      , nargs="+", help="Hidden sizes", default=[], type=int, required=True)
    parser.add_argument("--input_transform"   , help="Transformation to apply to inputs", type=str, default="none", choices=["binarize", "none", "tanh", "log1p"])
    # parser.add_argument("--input_size"      , help="Input size", type=int, default=None)
    # parser.add_argument("--tail_hidden_size"  , help="Tail Hidden size", default=0, type=int)
    parser.add_argument("--input_size_freq"   , help="Number of high importance features", type=int, default=None)
    parser.add_argument("--internal_batch_max", help="Maximum size of the internal batch", type=int, default=None)
    parser.add_argument("--last_dropout"      , help="Last dropout", type=float, default=0.2)
    parser.add_argument("--last_non_linearity", help="Last layer non-linearity", type=str, default="relu", choices=["relu", "tanh"])
    parser.add_argument("--lr"                , help="Learning rate", type=float, default=1e-3)
    parser.add_argument("--lr_alpha"          , help="Learning rate decay multiplier", type=float, default=0.3)
    parser.add_argument("--lr_steps"          , nargs="+", help="Learning rate decay steps", type=int, default=[10])
    parser.add_argument("--middle_dropout"    , help="Dropout for layers before the last", type=float, default=0.0)
    parser.add_argument("--middle_non_linearity", "--non_linearity", help="Before last layer non-linearity", type=str, default="relu", choices=["relu", "tanh"])
    parser.add_argument("--min_samples_class", help="Minimum number samples in each class and in each fold for AUC calculation (only used if aggregation_weight is not provided in --weights_class)", type=int, default=5)
    parser.add_argument("--min_samples_auc"  , help="Obsolete: use 'min_samples_class'", type=int, default=None)
    parser.add_argument("--min_samples_regr" , help="Minimum number of uncensored samples in each fold for regression metric calculation (only used if aggregation_weight is not provided in --weights_regr)", type=int, default=10)
    parser.add_argument("--normalize_loss"   , help="Normalization constant to divide the loss (default uses batch size)", type=float, default=None)
    parser.add_argument("--output_dir"       , help="Output directory, including boards (default 'models')", type=str, default="models")
    parser.add_argument("--prefix"           , help="Prefix for run name (default 'run')", type=str, default='run')
    parser.add_argument("--run_name"         , help="Run name for results", type=str, default=None)
    parser.add_argument("--save_model"       , help="Set this to 0 if the model should not be saved", type=int, default=1)
    parser.add_argument("--save_board"       , help="Set this to 0 if the TensorBoard should not be saved", type=int, default=1)
    parser.add_argument("--verbose"          , help="Verbosity level: 2 = full; 1 = no progress; 0 = no output", type=int, default=2, choices=[0, 1, 2])
    parser.add_argument("--weight_decay"     , help="Weight decay", type=float, default=0.0)
    parser.add_argument("--weights_class"    , "--task_weights", "--weights_classification"
                                             , help="CSV file with columns task_id, training_weight, aggregation_weight, task_type (for classification tasks)", type=str, default=None)
    parser.add_argument("--weights_regr"     , "--weights_regression"
                                             , help="CSV file with columns task_id, training_weight, censored_weight, aggregation_weight, aggregation_weight, task_type (for regression tasks)", type=str, default=None)
    return parser

In [2]:
def load_sparse(filename):
    """Loads sparse from Matrix market or Numpy .npy file."""
    if filename is None:
        return None
    if filename.endswith('.mtx'):
        return scipy.io.mmread(filename).tocsr()
    elif filename.endswith('.npy'):
        return np.load(filename, allow_pickle=True).item().tocsr()
    elif filename.endswith('.npz'):
        return scipy.sparse.load_npz(filename).tocsr()
    raise ValueError(f"Loading '{filename}' failed. It must have a suffix '.mtx', '.npy', '.npz'.")


In [3]:
def sparse_split2(tensor, split_size, dim=0):
    """
    Splits tensor into two parts.
    Args:
        split_size   index where to split
        dim          dimension which to split
    """
    assert tensor.layout == torch.sparse_coo
    indices = tensor._indices()
    values  = tensor._values()

    shape  = tensor.shape
    shape0 = shape[:dim] + (split_size,) + shape[dim+1:]
    shape1 = shape[:dim] + (shape[dim] - split_size,) + shape[dim+1:]

    mask0 = indices[dim] < split_size
    X0 = torch.sparse_coo_tensor(
            indices = indices[:, mask0],
            values  = values[mask0],
            size    = shape0)

    indices1       = indices[:, ~mask0]
    indices1[dim] -= split_size
    X1 = torch.sparse_coo_tensor(
            indices = indices1,
            values  = values[~mask0],
            size    = shape1)
    return X0, X1
